In [8]:
from collections import deque
from queue import PriorityQueue
import random
import math
import matplotlib.pyplot as plt
import time

In [9]:
class Problem:
    def __init__(self):
        self.init_state = None

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        raise NotImplementedError

    def step_cost(self, state, action):
        raise NotImplementedError

# Node class for search trees
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action)
        )

def solution(node):
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

# Sudoku problem class
class SudokuProblem(Problem):
    def __init__(self, init_state):
        super().__init__()
        self.init_state = init_state
        self.size = len(init_state)  # Dynamically determine grid size

    def actions(self, state):
        actions = []
        for row in range(self.size):
            for col in range(self.size):
                if state[row][col] == 0:  # Empty cell
                    for num in range(1, self.size + 1):
                        if self.is_valid_move(state, row, col, num):
                            actions.append((row, col, num))
        return actions

    def result(self, state, action):
        new_state = [row[:] for row in state]
        row, col, num = action
        new_state[row][col] = num
        return new_state

    def goal_test(self, state):
        for row in range(self.size):
            for col in range(self.size):
                if state[row][col] == 0:
                    return False
        return True

    def is_valid_move(self, state, row, col, num):
        # Check row
        if num in state[row]:
            return False
        # Check column
        if num in [state[r][col] for r in range(self.size)]:
            return False
        # Check subgrid
        subgrid_size = int(math.sqrt(self.size))
        start_row, start_col = subgrid_size * (row // subgrid_size), subgrid_size * (col // subgrid_size)
        for r in range(start_row, start_row + subgrid_size):
            for c in range(start_col, start_col + subgrid_size):
                if state[r][c] == num:
                    return False
        return True

    def step_cost(self, state, action):
        return 1

# Helper function to print Sudoku grid
def print_sudoku(grid):
    for row in grid:
        print(" ".join(str(num) if num != 0 else "." for num in row))
    print()

**BFS**

In [10]:
def breadth_first_search_with_tracking(problem):
    node = Node.root(problem.init_state)
    if problem.goal_test(node.state):
        return solution(node), []

    frontier = deque([node])
    explored = set()
    search_tree = []  # To track the parent-child relationships

    while frontier:
        node = frontier.popleft()
        explored.add(tuple(tuple(row) for row in node.state))

        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if tuple(tuple(row) for row in child.state) not in explored and child not in frontier:
                # Record the parent-child relationship
                search_tree.append((node.state, child.state))
                if problem.goal_test(child.state):
                    return solution(child), search_tree
                frontier.append(child)
    return None, search_tree


In [11]:
def main_fixed_v2():
    # Define a small Sudoku problem
    init_state = [
        [0, 2, 3],
        [3, 0, 0],
        [0, 0, 0],
    ]
    problem = SudokuProblem(init_state)

    # Solve using the updated BFS function
    (actions, states), search_tree = breadth_first_search_with_tracking(problem)

    # Visualize results
    print("Solution Actions:", actions)
    print("Solution States:")
    for state in states:
        for row in state:
            print(row)
        print()

    # Visualize search tree
    visualize_search_tree_fixed_v2(
        [(tuple(map(tuple, p)), tuple(map(tuple, c))) for p, c in search_tree],
        [tuple(map(tuple, state)) for state in states]
    )

main_fixed_v2()


Solution Actions: [(0, 0, 1), (1, 1, 1), (1, 2, 2), (2, 0, 2), (2, 1, 3), (2, 2, 1)]
Solution States:


TypeError: 'int' object is not iterable